If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
#! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\Admin\.cache\huggingface\token
Login successful


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [1]:
import transformers

print(transformers.__version__)

4.26.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [3]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_notebook", framework="pytorch")

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](images/causal_language_modeling.png)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](images/masked_language_modeling.png)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [2]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Found cached dataset wikitext (C:/Users/Admin/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 77.88it/s]


You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [4]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [3]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,text
0,= = = Player = = = \n
1,
2,
3,"At Torchwood 's Cardiff headquarters , Owen determines from an autopsy that the alien that impregnated Gwen is a Nostrovite , a race of carnivorous shape shifters who hunt in pairs and mate for life . He tells Jack that after fertilisation , the female transfers her eggs to the male who plants them in a host body until they are ready to hatch . Later , the female kills the host to release the offspring . At the wedding venue , the female Nostrovite ( Colette Brown ) murders and partly devours wedding guest Mervyn . Toshiko investigates along with Rhys ' best man "" Banana Boat "" ( Jonathan Lewis Owen ) and they are subsequently captured by the alien , who intends to feed on them later . As Gwen and Rhys prepare to say their wedding vows , Jack runs up the aisle and demands that the wedding be halted . Owen and Ianto ( Gareth David @-@ Lloyd ) free Toshiko but the half @-@ eaten corpse of Mervyn is discovered by his wife , a bridesmaid , who alerts the entire wedding party to the murder . The Nostrovite reveals herself and is chased away by Jack , escaping through a window . \n"
4,"The teeth are slender , cylindrical and tapering ( terete ) , less than 5 mm ( 0 @.@ 20 in ) long , and become shorter closer to the cap edge . They are crowded together , with typically between three and five teeth per square millimeter . Pinkish @-@ white initially , they age to a grayish @-@ brown . The stem is thick , very short , and often deformed . It becomes bulbous where it penetrates the ground , and may root into the soil for several centimeters . Although it may reach up to 5 cm ( 2 @.@ 0 in ) in total length , and is 1 to 3 cm ( 0 @.@ 4 to 1 @.@ 2 in ) wide , only about 0 @.@ 1 to 1 cm ( 0 @.@ 0 to 0 @.@ 4 in ) appear above ground . The upper part is covered with the same teeth found on the underside of the cap , whereas the lower part is hairy and often encases debris from the forest floor . The odor of the fruit body has been described as "" mild to disagreeable "" , or , as Banker suggested in his original description , similar to hickory nuts . \n"
5,"The arrival of a new owner in Lerner and of manager Martin O 'Neill marked the start of a new period of optimism at Villa Park and sweeping changes occurred throughout the club including a new badge , a new kit sponsor and team changes in the summer of 2007 . The first Cup final of the Lerner era came in 2010 when Villa were beaten 2 – 1 in the League Cup Final . Villa made a second trip to Wembley in that season losing 3 – 0 to Chelsea in the FA Cup semifinal . Just five days before the opening day of the 2010 – 11 season , O 'Neill resigned as manager with immediate effect . The club appointed Gérard Houllier as a replacement in September 2010 , but he stepped down on 1 June 2011 due to ill @-@ health . Houllier was replaced by Birmingham City manager Alex McLeish , despite numerous protests from fans against his appointment ; this was the first time that a manager had moved directly from Birmingham to Villa . McLeish 's contract was terminated at the end of the 2011 – 12 season after Villa finished in 16th place , only just above the relegation zone . On 2 July 2012 , Aston Villa confirmed the appointment of former Norwich City manager Paul Lambert as the replacement for McLeish . On 28 February 2012 , the club announced a financial loss of £ 53 @.@ 9 million . Lerner put the club up for sale on 12 May 2014 , with an estimated value of £ 200 million . With Lerner still on board , in the 2014 – 15 season Aston Villa scored just 12 goals in 25 league games , the lowest in Premier League history , and Lambert was sacked on 11 February 2015 . Tim Sherwood succeeded him , and saved Aston Villa from relegation while also leading them to the 2015 FA Cup Final , but he was sacked in the 2015 – 16 season , as was his successor Rémi Garde . Eric Black took temporary charge of the team , but was not able to prevent Villa from being relegated for the first

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [4]:
model_checkpoint = "distilgpt2"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [5]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [7]:
datasets["train"].column_names

['text']

In [9]:
tokenize_function(datasets["train"][10])

{'input_ids': [383, 983, 705, 82, 3344, 1080, 837, 262, 1086, 72, 51, 57, 1080, 837, 318, 5281, 625, 3264, 422, 569, 18354, 8704, 17740, 764, 5856, 10566, 837, 1938, 2922, 1123, 4326, 1262, 257, 1353, 2488, 12, 31, 866, 6650, 286, 262, 13480, 3975, 1058, 1752, 257, 2095, 318, 6163, 837, 262, 2137, 6100, 262, 2095, 1088, 262, 13480, 287, 2368, 2488, 12, 31, 1048, 764, 317, 2095, 460, 691, 719, 1752, 583, 2488, 12, 31, 1210, 837, 475, 3435, 460, 307, 7520, 3294, 4962, 379, 262, 10907, 286, 584, 3435, 705, 4962, 764, 5501, 2095, 468, 257, 2214, 290, 5253, 286, 3356, 3614, 416, 511, 7561, 35094, 469, 764, 3205, 284, 5193, 3435, 460, 307, 8686, 284, 257, 2060, 4365, 764, 5856, 11327, 837, 3435, 481, 869, 503, 611, 1223, 4325, 284, 606, 837, 884, 355, 511, 1535, 2173, 357, 6574, 1267, 1972, 1877, 393, 852, 13642, 503, 416, 4472, 3434, 764, 5501, 2095, 468, 2176, 366, 6902, 14817, 366, 837, 4678, 3748, 284, 1123, 2095, 764, 1119, 389, 9086, 656, 366, 15644, 32480, 366, 837, 543, 389, 28690, 4

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [8]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-7e08abd08b7825dd.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-9216adbba228627f.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-2bcee6e0178df6b6.arrow


In [9]:
print(type(tokenized_datasets))
tokenized_datasets["train"][0]
tokenized_datasets

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [10]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [11]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [12]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=1,
)

Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-5f6dc90dada7f050.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-0f28ab7d017bfbea.arrow
Loading cached processed dataset at C:\Users\Admin\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126\cache-01f106e266a00bf9.arrow


And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [14]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Oz'

In [14]:
len(lm_datasets["train"][2]["input_ids"])
print(lm_datasets["train"][2])

{'input_ids': [6909, 764, 317, 1588, 1074, 286, 8786, 12118, 262, 4226, 764, 383, 983, 705, 82, 4756, 7505, 373, 23568, 416, 1737, 705, 77, 764, 220, 198, 632, 1138, 351, 3967, 4200, 287, 2869, 837, 290, 373, 15342, 416, 1111, 4960, 290, 8830, 9188, 764, 2293, 2650, 837, 340, 2722, 41496, 2695, 837, 1863, 351, 281, 9902, 8313, 287, 3389, 286, 326, 614, 764, 632, 373, 635, 16573, 656, 15911, 290, 281, 2656, 2008, 11034, 2168, 764, 14444, 284, 1877, 4200, 286, 569, 18354, 7496, 17740, 2873, 837, 569, 18354, 7496, 17740, 6711, 373, 407, 36618, 837, 475, 257, 4336, 11059, 11670, 351, 262, 983, 705, 82, 9902, 8313, 373, 2716, 287, 1946, 764, 6343, 13, 44206, 561, 1441, 284, 262, 8663, 351, 262, 2478, 286, 569, 18354, 7496], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [16]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [17]:
from transformers import Trainer, TrainingArguments

In [18]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [20]:
trainer.train()

c:\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18667
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3501
  Number of trainable parameters = 81912576
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anforsm (dt2112g1). Use `wandb login --relogin` to force relogin


  0%|          | 0/3501 [00:00<?, ?it/s]c:\Python38\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Python38\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
 14%|█▍        | 500/3501 [01:50<10:03,  4.97it/s]Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-500
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-500\config.json
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-500\generation_config.json


{'loss': 3.9455, 'learning_rate': 1.7143673236218227e-05, 'epoch': 0.43}


Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-500\pytorch_model.bin
c:\Python38\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Python38\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
 29%|██▊       | 1000/3501 [03:33<08:30,  4.90it/s]Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-1000
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-1000\config.json
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-1000\generation_config.json


{'loss': 3.8053, 'learning_rate': 1.4287346472436447e-05, 'epoch': 0.86}


...

Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-3000\pytorch_model.bin
c:\Python38\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Python38\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|█████████▉| 3500/3501 [12:29<00:00,  4.91it/s]Saving model checkpoint to distilgpt2-finetuned-wikitext2\checkpoint-3500
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-3500\config.json
Configuration saved in distilgpt2-finetuned-wikitext2\checkpoint-3500\generation_config.json


{'loss': 3.6398, 'learning_rate': 5.712653527563554e-09, 'epoch': 3.0}


Model weights saved in distilgpt2-finetuned-wikitext2\checkpoint-3500\pytorch_model.bin
c:\Python38\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Python38\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
100%|██████████| 3501/3501 [12:31<00:00,  1.64it/s]***** Running Evaluation *****
  Num examples = 1930
  Batch size = 16
                                                   
100%|██████████| 3501/3501 [12:41<00:00,  1.64it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 3501/3501 [12:41<00:00,  4.60it/s]

{'eval_loss': 3.654172897338867, 'eval_runtime': 10.4752, 'eval_samples_per_second': 184.244, 'eval_steps_per_second': 11.551, 'epoch': 3.0}
{'train_runtime': 765.7383, 'train_samples_per_second': 73.133, 'train_steps_per_second': 4.572, 'train_loss': 3.733408645097612, 'epoch': 3.0}


TrainOutput(global_step=3501, training_loss=3.733408645097612, metrics={'train_runtime': 765.7383, 'train_samples_per_second': 73.133, 'train_steps_per_second': 4.572, 'train_loss': 3.733408645097612, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [36]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1930
  Batch size = 8
100%|██████████| 242/242 [00:07<00:00, 32.31it/s]

Perplexity: 38.23


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [ ]:
model_checkpoint = "distilroberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-333e4baa6f280a66.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-23acd0930cc16da7.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-56ae8ad41a9fdf19.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-599a47a0e666ad65.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cach

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-661796332aa2b576.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-e019d91824c225fd.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-b5875c725d0e5cb7.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-a8e3eeaa703ca023.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cach

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.109144,1.919501
2,1.986566,1.884295
3,1.952879,1.861102


TrainOutput(global_step=7218, training_loss=2.0377309222603213)

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.37


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```